In [18]:
import pandas as pd

# Cargar los CSV correctamente con separador ;
df_recopilados = pd.read_csv("datos_recopilados.csv", sep=';')
df_pronostico = pd.read_csv("datos_pronostico.csv", sep=';')


In [19]:
from sklearn.impute import SimpleImputer

# Variables predictoras
features = [
    'TMAX', 'TMIN', 'PREC',
]

# Separar datos X (features) e y (target)
X = df_recopilados[features]
y = df_recopilados['RIEGO']

# Imputar valores faltantes con la media
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Crear y entrenar modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.93      0.93      0.93     29188
           1       0.76      0.78      0.77      8536

    accuracy                           0.89     37724
   macro avg       0.85      0.85      0.85     37724
weighted avg       0.90      0.89      0.90     37724



In [21]:
importances = model.feature_importances_
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance)


  Feature  Importance
0    TMAX    0.605468
1    TMIN    0.352038
2    PREC    0.042494


In [22]:
# Preparar datos de pronóstico
X_pred = imputer.transform(df_pronostico[features])

# Predecir riego
df_pronostico['RIEGO_PREDICT'] = model.predict(X_pred)

# Mostrar los resultados
print(df_pronostico[['PARAJE', 'MES', 'FECHA', 'RIEGO_PREDICT']].head())


     PARAJE    MES       FECHA  RIEGO_PREDICT
0  Abanilla  Enero  01/01/2025              0
1  Abanilla  Enero  02/01/2025              0
2  Abanilla  Enero  03/01/2025              0
3  Abanilla  Enero  04/01/2025              0
4  Abanilla  Enero  05/01/2025              0


In [23]:
df_pronostico.to_csv("datos_pronostico_con_riego.csv", index=False, sep=';')
